In [16]:
import pennylane as qml
from pennylane import numpy as pnp
from scipy.optimize import differential_evolution
from scipy.stats.qmc import Halton
import os
import json
import numpy as np
from datetime import datetime
import time
from qiskit.quantum_info import SparsePauliOp
from susy_qm import calculate_Hamiltonian

In [53]:
shots = None
potential = 'QHO'
cutoff = 16

In [54]:
starttime = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
folder = str(starttime)

base_path = os.path.join(r"C:\Users\Johnk\Documents\PhD\Quantum Computing Code\Quantum-Computing\SUSY\SUSY QM\PennyLane\SSVQE\Files", potential, folder)
os.makedirs(base_path, exist_ok=True)

In [55]:
H = calculate_Hamiltonian(cutoff, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])[:3]
min_eigenvalue = min(eigenvalues.real)

hamiltonian = SparsePauliOp.from_operator(H)
num_qubits = hamiltonian.num_qubits

In [56]:
input_states = []
for j in range(len(eigenvalues)):
    state = np.zeros(2**num_qubits)
    state[j] = 1.0
    input_states.append(state)

In [64]:
input_states = []
degeneracies = {0: 1, 1: 2}  # Example for QHO
for eigenvalue, count in degeneracies.items():
    for _ in range(count):
        state = np.zeros(2**num_qubits)
        state[len(input_states)] = 1.0
        input_states.append(state)

In [71]:
num_layers = 1
params_shape = qml.StronglyEntanglingLayers.shape(n_layers=num_layers, n_wires=num_qubits)

def ansatz(params, wires):
    params = pnp.tensor(params.reshape(params_shape), requires_grad=True)
    qml.StronglyEntanglingLayers(weights=params, wires=wires, imprimitive=qml.CZ)

dev = qml.device('default.qubit', wires=num_qubits, shots=shots)

@qml.qnode(dev)
def expected_value(params, phi):
    qml.StatePrep(phi, wires=range(num_qubits))
    ansatz(params, wires=range(num_qubits))
    return qml.expval(qml.Hermitian(H, wires=range(num_qubits)))


def loss_f_ssvqe(params):
    cost = 0
    for phi in input_states:
        cost += expected_value(params, phi)
    return cost

def loss_f_weighted_ssvqe(params, weights):
    cost = 0
    for phi, weight in zip(input_states, weights):
        cost += weight * expected_value(params, phi)
    return cost

penalty_weight = 1#00.0
def loss_f_weighted_ssvqe(params, weights):
    cost = 0
    for i, phi in enumerate(input_states):
        weight = weights[i]
        exp_value = expected_value(params, phi)
        cost += weight * exp_value
        
        # Add orthogonality penalty within degenerate subspaces
        for j in range(i):
            if np.isclose(eigenvalues[i], eigenvalues[j]):
                overlap = np.abs(np.vdot(input_states[i], input_states[j]))**2
                cost += penalty_weight * overlap
    return cost



In [72]:
bounds = [(0, 2 * np.pi) for _ in range(np.prod(params_shape))]
max_iter = 10000
popsize = 20
tol = 1e-3
abs_tol = 1e-3

weights = np.arange(len(input_states), 0, -1)

seed = (os.getpid() * int(time.time())) % 123456789
halton_sampler = Halton(d=np.prod(params_shape), seed=seed)
halton_samples = 2 * np.pi * halton_sampler.random(n=popsize)

print("Starting SSVQE optimization")
#res = differential_evolution(loss_f_ssvqe, bounds, maxiter=max_iter, tol=tol, atol=abs_tol, popsize=popsize, init=halton_samples, seed=seed)
res = differential_evolution(
    lambda params: loss_f_weighted_ssvqe(params, weights),
    bounds,
    maxiter=max_iter,
    tol=tol,
    atol=abs_tol,
    popsize=popsize,
    init=halton_samples,
    seed=seed
)

optimized_params = res.x
optimized_energies = [expected_value(optimized_params, phi) for phi in input_states]

print("Optimization complete")

Starting SSVQE optimization
Optimization complete


In [73]:
run = {
    "starttime": starttime,
    "potential": potential,
    "cutoff": cutoff,
    "exact_eigenvalues": [x.real.tolist() for x in eigenvalues],
    "ansatz": "StronglyEntanglingLayers-1layer",
    "optimized_energies": [float(e) for e in optimized_energies],
    "params": optimized_params.tolist(),
    "success": res.success,
    "iterations": res.nit,
    #"total_run_time": str(res.execution_time),
}

run


{'starttime': '2025-01-15_20-06-44',
 'potential': 'QHO',
 'cutoff': 16,
 'exact_eigenvalues': [0.0, 1.0, 1.0000000000000002],
 'ansatz': 'StronglyEntanglingLayers-1layer',
 'optimized_energies': [6.685145590978086e-14,
  1.0000000000000673,
  2.0000000000000675],
 'params': [5.281501258616705,
  3.1415921364768233,
  6.204031526931203,
  3.0906227926171352,
  0.0,
  4.182437964817821,
  4.215893570810944,
  0.0,
  2.2562228206531545,
  0.42627803562418465,
  6.283185307179586,
  6.233778099403259,
  3.5779102598573553,
  0.0,
  0.9386252513003522],
 'success': True,
 'iterations': 48}

In [ ]:
path = os.path.join(base_path, "{}_{}.json".format(potential, cutoff))
with open(path, 'w') as json_file:
    json.dump(run, json_file, indent=4)

print(f"Results saved to {path}")